# Agent 3: Retrieve Knowledge

Retrieve knowledge is an agent which uses vector search to find relevant information from internal knowledge base

In [1]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create a client to interact with the Azure AI service
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

## Define knowledge base

In [5]:
# Upload file and create vector store
file = project_client.agents.upload_file_and_poll(
    file_path="./data/Energy Saving Tips.pdf", purpose="assistants")
print(f"Uploaded file, file ID: {file.id}")

vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-QWDLK1ufXoOdCqto3Z4V5WhJ
Created vector store, vector store ID: vs_7MewlV44Pks96e7B357kes7I


## Create Agent

In [9]:
from azure.ai.projects.models import (
    FileSearchTool,
)

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

retrieve_agent= project_client.agents.create_agent(
    model=os.environ["AZURE_OPENAI_DEPLOYMENT"],
    name="energy_saving_info",
    description="The Energy saving tips suggests actions to take to save on utilities bill.",
    instructions="Hello, you are helpful assistant and can search information from uploaded files.Only use the uploaded information to answer questions.",
    tools=file_search.definitions,
    tool_resources=file_search.resources,
    # Parameters
    temperature=0.7,
    top_p=0.95,
)

print(f"Created agent, agent ID: {retrieve_agent.id}")

Created agent, agent ID: asst_kft56ugjtndPNGvX6Cdszk9T


In [10]:
# Create Thread
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="I do heavy cooking, how can I save electricity in such a case?"
)
print(f"Created message, ID: {message.id}")

# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, assistant_id=retrieve_agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Created thread, ID: thread_hevky7VdO20nWuooNM7ijMpU
Created message, ID: msg_xkzeI2eb0B7ALqMWM4HGvogX
Run finished with status: RunStatus.COMPLETED


In [ ]:
# Helper function

from datetime import datetime
def get_conversation_md(conversation):
    """
    Function to return a conversation in Markdown (MD) format as a string.
    """
    messages = conversation.get("data", [])
    if not messages:
        return "No messages found in the conversation."

    # Initialize a list to hold the Markdown lines
    md_lines = []
    md_lines.append("# Conversation")
    md_lines.append("___")  # Markdown horizontal line

    # Iterate through the messages
    # Reversing to maintain chronological order
    for message in reversed(messages):
        role = message.get("role", "unknown").capitalize()
        timestamp = message.get("created_at")
        content = message.get("content", [])

        # Convert timestamp to a readable format
        if timestamp:
            timestamp = datetime.fromtimestamp(
                timestamp).astimezone().strftime('%Y-%m-%d %H:%M:%S %Z')
        else:
            timestamp = "Unknown time"

        # Extract the text content
        message_text = ""
        for item in content:
            if item.get("type") == "text":
                message_text += item["text"].get("value", "")

        # Append the message in Markdown format
        md_lines.append(f"### **{role}** ({timestamp})")
        md_lines.append(f"{message_text}")
        md_lines.append("___")  # Markdown horizontal line

    # Join the lines with newlines to form the complete Markdown string
    return "\n".join(md_lines)

In [11]:
# Display all messages in the thread
from IPython.display import Markdown, display

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(get_conversation_md(messages)))

# Conversation
___
### **User** (2025-02-06 18:11:04 Malay Peninsula Standard Time)
I do heavy cooking, how can I save electricity in such a case?
___
### **Assistant** (2025-02-06 18:11:07 Malay Peninsula Standard Time)
To save electricity while doing heavy cooking, you can follow these tips:

1. **Cover Pots While Cooking**: Food cooks faster in covered pots, which saves energy.
2. **Match Pot Size to Burner Size**: Using a pot that matches the size of the burner prevents energy loss.
3. **Use Microwave for Leftovers**: A microwave oven uses less energy compared to a conventional oven for cooking or warming leftovers.
4. **Use an Oven Timer**: Instead of opening the oven door to check your food, use an oven timer to avoid losing heat.
5. **Clean Appliances Regularly**: Regular cleaning of cooking appliances ensures efficient heat transfer.
6. **Check Oven Door Seal**: Ensure that the seal on your oven door is intact to retain heat more effectively.
7. **Plan Cooking Procedures**: Prepare your ingredients and plan your cooking steps ahead of time to minimize energy usage【4:0†Energy Saving Tips】.

By implementing these practices, you can reduce your electricity consumption even while engaging in heavy cooking activities.
___

In [12]:
# Print citations from the messages
for citation in messages.file_citation_annotations:
    print(citation)

{'type': 'file_citation', 'text': '【4:0†Energy Saving Tips】', 'start_index': 867, 'end_index': 891, 'file_citation': {'file_id': 'assistant-QWDLK1ufXoOdCqto3Z4V5WhJ'}}


In [ ]:
# Delete the file when done
project_client.agents.delete_vector_store(vector_store.id)
print("Deleted vector store")

project_client.agents.delete_file(file_id=file.id)
print("Deleted file")

# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")